In [ ]:
import pandas as pd

In [ ]:
MODEL = "distilbert-base-uncased"
BATCH_SIZE = 4

In [105]:
columns=["label", "text", "author", "subreddit", "score", "ups", "downs", "date", "created_utc", "parent_comment"]
train_data_df = pd.read_csv("train-balanced.csv", delimiter="\t", names=columns, header=None)
test_data_df = pd.read_csv("test-balanced.csv", delimiter="\t", names=columns, header=None)

train_data_df.drop(columns=["author", "subreddit", "score", "ups", "downs", "date", "created_utc", "parent_comment"],axis=1, inplace=True)
test_data_df.drop(columns=["author", "subreddit", "score", "ups", "downs", "date", "created_utc", "parent_comment"],axis=1, inplace=True)

In [106]:
train_data_df = train_data_df.dropna()
test_data_df = test_data_df.dropna()

In [115]:
train_data_df.to_csv("train.csv", sep='\t')

In [116]:
test_data_df.to_csv("test.csv", sep='\t')

In [117]:
from datasets import load_dataset
# sarc_train = load_dataset('csv', data_files=['train-balanced.csv'], delimiter=",", split="train",on_bad_lines='skip')
# print(sarc_train[0])
# columns=["label", "text", "author", "subreddit", "score", "ups", "downs", "date", "created_utc", "parent_comment"]
columns=["label", "text"]

data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset('csv',data_files=data_files, delimiter="\t",column_names=columns, )

Using custom data configuration default-322defad502acf09


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\fahad\.cache\huggingface\datasets\csv\default-322defad502acf09\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# dataset = dataset.remove_columns(["author", "subreddit", "score", "ups", "downs", "date", "created_utc", "parent_comment"])

In [118]:
dataset["train"][35431]["text"]

'But blacking out is fun'

In [119]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [120]:
def preprocess_function(examples):
    try:
        return tokenizer(examples["text"], truncation=True)
    except TypeError as t:
        for idx, item in enumerate(examples['text']):
            if not isinstance(item, str):
                print(idx - 1, examples['text'][idx - 1])
                print(idx, item)
                print(idx + 1, examples['text'][idx + 1])
            
        #print(*examples["text"])
        raise t

In [121]:
tokenizer("test", truncation=True)

{'input_ids': [101, 3231, 102], 'attention_mask': [1, 1, 1]}

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, disable_nullable=True)

  0%|          | 0/1126 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)

In [ ]:
from transformers import TrainingArguments, Trainer
from os import path

output_dir_path = path.join("results")
training_args = TrainingArguments(
    output_dir = output_dir_path,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()